In [ ]:
!rm -r sample_data/
!wget https://www.thetrash.altervista.org/HPC/ML/archive.zip
!unzip archive.zip

## Import delle varie librerie e del dataset

In [ ]:
import time
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

import numpy as np                      # advanced math library
import matplotlib.pyplot as plt         # MATLAB like plotting routines
import random                           # for generating random numbers

from keras.models import Sequential, Model, load_model  # Model type to be used

from keras.layers.core import Dense, Dropout, Activation
                                        # Types of layers to be used in
                                        # our model
from keras.utils import np_utils        # NumPy related tools

from keras.preprocessing import image_dataset_from_directory, image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import (Conv2D, MaxPooling2D, Flatten)
from keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16

from IPython.display import Image

from numpy.random import seed
seed(56465423)

In [ ]:
#width height
w, h  = (128,128)
# w, h  = (224,224) # Terzo modello
#batch size
b = 16 # Primo modello
#b = 32 # secondo modello
#b = 64 # terzo modello

#import del dataset
train_path = 'training_set/training_set'
test_path = 'test_set/test_set'

train_datagen = ImageDataGenerator( rescale = 1./ 255, shear_range = 0.2,
                                   zoom_range = 0.2, horizontal_flip = True) 
                
test_datagen = ImageDataGenerator( rescale = 1. / 255 ) 

# path to image , resize dimension, batch size
train_generator = train_datagen.flow_from_directory(train_path, 
                                                    target_size =(w, h),
                                                    batch_size = b,
                                                    class_mode ='binary') 
  
validation_generator = test_datagen.flow_from_directory( test_path, 
                                                        target_size =(w, h), 
                                                        batch_size = b, 
                                                        class_mode ='binary')

print("\n")
print("Lunghezza del train generator >",len(train_generator),"\nShape > ",train_generator[0][0].shape )
print("Lunghezza del validation generator >",len(validation_generator),"\nShape > ",validation_generator[0][0].shape)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


Lunghezza del train generator > 501 
Shape >  (16, 128, 128, 3)
Lunghezza del validation generator > 127 
Shape >  (16, 128, 128, 3)


## Dichiarazione dei modelli

### Modelli:

#### Primo modello 
* conv: 32-> 32-> 64 -> flatten -> dense 64 -> dense 1

widht and height = 128

batch size = 16

epochs = 20

In [ ]:
model_name = "first"
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(w,h,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#Model compilation
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

model.summary()

#### Secondo Modello
* conv: 32-> 64-> 128 -> flatten -> dense 128 -> dense 1

widht and height = 128

batch size = 32

In [ ]:
model_name = "second"
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(w,h,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#Model compilation
sgd = SGD(lr=0.001 ,  momentum =0.9)
model.compile(loss='binary_crossentropy', optimizer= sgd, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0

#### Terzo Modello
* VGG 16 -> flatten -> dense 128 -> dense 1

widht and height = 224

batch size = 64

In [ ]:
model_name = "third"
model = VGG16(include_top=False, input_shape=(w, h, 3))
# mark loaded layers as not trainable
for layer in model.layers:
	layer.trainable = False
	# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
	# define new model
model = Model(inputs=model.inputs, outputs=output)
	# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Fitting Del Modello

In [ ]:
print(f'starting of the {model_name} model')
start_time = time.time()
history = model.fit(train_generator, 
                    steps_per_epoch = len(train_generator), epochs = 40, 
                    validation_data = (validation_generator), 
                    validation_steps= len(validation_generator))

end_time = time.time() - start_time
print("Total train time:",time.gmtime(end_time).tm_hour, "h", time.gmtime(end_time).tm_min,"m",  time.gmtime(end_time).tm_sec,"s")

# evaluate model
_, acc = model.evaluate(validation_generator, steps=len(validation_generator), verbose=0)
print('accuracy > %.3f ' % (acc * 100.0))
#Salvataggio del modello
model.save(f'{model_name}_model.h5') 

## Plot

In [ ]:
 # plot loss
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.legend()
plt.savefig('loss_plot.png')
plt.close()

# plot accuracy
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.legend()
plt.savefig('accuracy_plot.png')
plt.close()

In [ ]:
#plot del modello
from keras.utils.vis_utils import plot_model
plot_model(model, to_file=f'{model_name}_model_plot.png', show_shapes=True, show_layer_names=True)

## Immagini di test

* di un gatto
* di un gatto con rumore
* di un gatto generico preso da internet
* di un cane
* di un cane coperto in parte
* di un cane generico preso da internet

In [ ]:
!mkdir img
%cd img

!wget http://www.thetrash.altervista.org/HPC/ML/dog.jpg
!wget http://www.thetrash.altervista.org/HPC/ML/dog_luca.jpg
!wget http://www.thetrash.altervista.org/HPC/ML/dog_generic.jpg
!wget http://www.thetrash.altervista.org/HPC/ML/cat.jpg
!wget http://www.thetrash.altervista.org/HPC/ML/cat_meme.png
!wget http://www.thetrash.altervista.org/HPC/ML/cat_filippo.jpg
%cd ..

## Validation multipla


In [ ]:
#model = load_model(f'{model_name}_model.h5')
model = load_model('third_model.h5')
for filename in os.listdir('img'):
  test_image = image.load_img('img/'+filename, target_size = (w,h))
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image, axis = 0)
  result = model.predict(test_image, verbose=2)
#{'cats': 0, 'dogs': 1}
  if result == 1:
    prediction = 'dog'
  else:
    prediction = 'cat'

  print(f'my prediction on {filename} is {prediction}')

1/1 - 0s
my prediction on cat_filippo.jpg is dog with 100% of accuracy
1/1 - 0s
my prediction on dog_luca.jpg is dog with 100% of accuracy
1/1 - 0s
my prediction on dog_generic.jpg is dog with 100% of accuracy
1/1 - 0s
my prediction on cat_meme.png is cat with 100% of accuracy
1/1 - 0s
my prediction on cat.jpg is cat with 100% of accuracy
1/1 - 0s
my prediction on dog.jpg is dog with 100% of accuracy
